In [2]:
import pandas as pd
import time
import numpy as np
from nba_api.stats.endpoints import playercareerstats
import nba_api.stats.endpoints as nbaapi
from nba_api.stats.endpoints import leaguegamefinder
import psycopg2
import re
import allintwo

In [5]:
# from nba_api.stats.endpoints import leaguegamefinder

conn = allintwo.connect_to_rds('thebigone', 'ajwin', 'CharlesBark!23', 'nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com')

seasons = []
for i in list(range(1983,2025)):
    seasons.append(str(i)+'-'+str(i+1)[2:])
seasons = seasons[::-1]

## Takes only 1:20 to pull
gamesl = []
for i in seasons:
    gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable=i).get_data_frames()[0]
    gamesl.append(gamefinder)
    time.sleep(1)
gamehistory = pd.concat(gamesl,axis=0)
gamehistory['GAME_DATE'] =  pd.to_datetime(gamehistory['GAME_DATE'])

## Takes only 1:07
playersl = []
for i in seasons:
    playerfinder = nbaapi.leaguedashplayerbiostats.LeagueDashPlayerBioStats(season=i).get_data_frames()[0]
    playerfinder['season']=i
    playersl.append(playerfinder)
    time.sleep(.5)
allplayers = pd.concat(playersl,axis=0)

##### THIS PROCESSES AND RECREATES THE MASTER TABLES 
### DROP TABLES
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS mastergames;")
cursor.execute("DROP TABLE IF EXISTS masterplayers;")

conn.commit()

gamehistory = allintwo.clean_column_names(gamehistory)
allplayers = allintwo.clean_column_names(allplayers)

allintwo.create_table(conn,'mastergames',gamehistory)
allintwo.create_table(conn,'masterplayers',allplayers)

allintwo.insert_dataframe_to_rds(conn,
    gamehistory,
    table_name='mastergames'
)

allintwo.insert_dataframe_to_rds(conn,
    allplayers,
    table_name='masterplayers'
)


Connected to RDS PostgreSQL database


C:\Users\ajwin\AppData\Local\Temp\ipykernel_417328\4277887209.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gamehistory = pd.concat(gamesl,axis=0)
C:\Users\ajwin\AppData\Local\Temp\ipykernel_417328\4277887209.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  allplayers = pd.concat(playersl,axis=0)


Master table mastergames created successfully.
Master table masterplayers created successfully.
Data inserted successfully into mastergames table.
Data inserted successfully into masterplayers table.


In [6]:
conn = allintwo.connect_to_rds('thebigone', 'ajwin', 'CharlesBark!23', 'nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com')
g1 = allintwo.fetch_table_to_dataframe(conn,'mastergames')

Connected to RDS PostgreSQL database
Data fetched successfully from mastergames table.


In [7]:
# Call the function, passing the existing connection
g1

,seasonid,teamid,teamabbreviation,teamname,gameid,gamedate,matchup,wl,min,pts,...,ftpct,oreb,dreb,reb,ast,stl,blk,tov,pf,plusminus
0,22024,1610612748,MIA,Miami Heat,0022400065,2024-10-23,MIA vs. ORL,None,121,54,...,0.714,7.0,20.0,27,13,2.0,5,4,10,-4.0
1,22024,1610612755,PHI,Philadelphia 76ers,0022400066,2024-10-23,PHI vs. MIL,None,120,61,...,0.833,13.0,22.0,35,10,2.0,1,4,16,-11.0
2,22024,1610612766,CHA,Charlotte Hornets,0022400068,2024-10-23,CHA @ HOU,None,120,49,...,0.692,6.0,17.0,23,11,2.0,6,7,7,-5.8
3,22024,1610612741,CHI,Chicago Bulls,0022400069,2024-10-23,CHI @ NOP,None,120,58,...,0.833,5.0,19.0,24,12,0.0,0,9,9,-1.0
4,22024,1610612739,CLE,Cleveland Cavaliers,0022400067,2024-10-23,CLE @ TOR,None,133,83,...,0.875,6.0,21.0,27,18,7.0,8,7,15,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123115,21983,1610612757,POR,Portland Trail Blazers,0028300007,1983-10-28,POR vs. SDC,W,240,107,...,0.862,9.0,27.0,36,29,7.0,6,17,27,NaN
123116,21983,1610612758,KCK,Kansas City Kings,0028300002,1983-10-28,KCK vs. LAL,L,240,107,...,0.739,21.0,24.0,45,17,5.0,6,22,25,NaN
123117,21983,1610612739,CLE,Cleveland Cavaliers,0028300008,1983-10-28,CLE vs. NYK,L,240,106,...,0.650,18.0,17.0,35,22,9.0,8,18,25,NaN
123118,21983,1610612743,DEN,Denver Nuggets,0028300009,1983-10-28,DEN vs. UTH,W,240,139,...,0.744,19.0,37.0,56,40,4.0,8,11,26,NaN


In [1]:
x = [1,2,3,4,5]
y = [1,2,3]

In [3]:
set.difference(x)

TypeError: descriptor 'difference' for 'set' objects doesn't apply to a 'list' object

In [4]:
difference = set(x) - set(y)
result = list(difference)

In [5]:
result

[4, 5]